In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.applications import ResNet50V2

In [2]:
# Function to count classes in the dataset
def Classes_Count(path, name):
    Classes_Dict = {}
    for Class in os.listdir(path):
        Full_Path = os.path.join(path, Class)
        Classes_Dict[Class] = len(os.listdir(Full_Path))
    df = pd.DataFrame(Classes_Dict, index=[name])
    return df

In [7]:
# Data paths
train_dir = 'F:\\PG\\Semester 2\\MP II\\Dataset\\train\\'
test_dir = 'F:\\PG\\Semester 2\\MP II\\Dataset\\test\\'

In [8]:
# Count classes in train and test datasets
Train_Count = Classes_Count(train_dir, 'train').transpose().sort_values(by="train", ascending=False)
Test_Count = Classes_Count(test_dir, 'test').transpose().sort_values(by="test", ascending=False)
pd.concat([Train_Count, Test_Count], axis=1)

,train,test
happy,7215,1774
neutral,4965,1233
sad,4830,1247
fear,4097,1024
angry,3995,958
surprise,3171,831
disgust,436,111


In [34]:
# Image parameters
img_shape = 224
batch_size = 64

In [35]:
# ImageDataGenerator for train and test data
train_preprocessor = ImageDataGenerator(
    rescale=1 / 255.,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
)

test_preprocessor = ImageDataGenerator(rescale=1 / 255.)

train_data = train_preprocessor.flow_from_directory(
    train_dir,
    class_mode="categorical",
    target_size=(img_shape, img_shape),
    color_mode='rgb',
    shuffle=True,
    batch_size=batch_size,
    subset='training',
)

test_data = test_preprocessor.flow_from_directory(
    test_dir,
    class_mode="categorical",
    target_size=(img_shape, img_shape),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size,
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [36]:
# Load pre-trained ResNet50V2 model
ResNet50V2_base = ResNet50V2(input_shape=(img_shape, img_shape, 3),
                             include_top=False,
                             weights='imagenet'
                             )

# Freeze all layers except last 50
ResNet50V2_base.trainable = True
for layer in ResNet50V2_base.layers[:-50]:
    layer.trainable = False

# Custom model using ResNet50V2
def Create_Custom_ResNet50V2_Model():
    model = Sequential([
        ResNet50V2_base,
        Dropout(0.25),
        BatchNormalization(),
        Flatten(),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(7, activation='softmax')
    ])
    return model

ResNet50V2_Model = Create_Custom_ResNet50V2_Model()
ResNet50V2_Model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [37]:
# Callbacks
checkpoint_path = "ResNet50V2_Model_Checkpoint.keras"
Checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True)
Early_Stopping = EarlyStopping(monitor='val_accuracy', patience=7, restore_best_weights=True, verbose=1)
Reducing_LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1)

callbacks = [Early_Stopping, Reducing_LR]

steps_per_epoch = train_data.n // train_data.batch_size
validation_steps = test_data.n // test_data.batch_size

In [38]:
ResNet50V2_history = ResNet50V2_Model.fit(train_data, validation_data=test_data, epochs=30, batch_size=batch_size,
                                           callbacks=callbacks, steps_per_epoch=steps_per_epoch,
                                           validation_steps=validation_steps)

Epoch 1/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 4249s 9s/step - accuracy: 0.4121 - loss: 1.8249 - val_accuracy: 0.5582 - val_loss: 1.1871 - learning_rate: 0.0010
Epoch 2/30
  1/448 ━━━━━━━━━━━━━━━━━━━━ 50:30 7s/step - accuracy: 0.5000 - loss: 1.4981

C:\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


448/448 ━━━━━━━━━━━━━━━━━━━━ 14s 16ms/step - accuracy: 0.5000 - loss: 1.4981 - val_accuracy: 0.7000 - val_loss: 0.6085 - learning_rate: 0.0010
Epoch 3/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 29054s 65s/step - accuracy: 0.5547 - loss: 1.2321 - val_accuracy: 0.5862 - val_loss: 1.0966 - learning_rate: 0.0010
Epoch 4/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.6250 - loss: 1.1920 - val_accuracy: 1.0000 - val_loss: 0.2553 - learning_rate: 0.0010
Epoch 5/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 4072s 9s/step - accuracy: 0.5508 - loss: 1.2721 - val_accuracy: 0.5968 - val_loss: 1.0737 - learning_rate: 0.0010
Epoch 6/30
  1/448 ━━━━━━━━━━━━━━━━━━━━ 58:18 8s/step - accuracy: 0.5000 - loss: 1.2789
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
448/448 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.5000 - loss: 1.2789 - val_accuracy: 0.8000 - val_loss: 0.4703 - learning_rate: 0.0010
Epoch 7/30
448/448 ━━━━━━━━━━━━━━━━━━━━ 5059s 11s/step - accuracy: 0.6031 - loss: 1.097

In [39]:
ResNet50V2_Score = ResNet50V2_Model.evaluate(test_data)
print("Test Loss: {:.5f}".format(ResNet50V2_Score[0]))
print("Test Accuracy: {:.2f}%".format(ResNet50V2_Score[1] * 100))


113/113 ━━━━━━━━━━━━━━━━━━━━ 476s 4s/step - accuracy: 0.4240 - loss: 1.4456
Test Loss: 1.09750
Test Accuracy: 58.60%
